### Below shows how much non-empty data is available in each table  

In [8]:
import pandas as pd

# List of all tables names available in subject tables
df_tables = pd.read_csv('all_subject_tables.csv')

# Initialize an empty DataFrame to store counts
data_counts = pd.DataFrame()
years = range(2015, 2023)

# Initialize a temporary list to hold data before appending to DataFrame
temp_data = []

# Loop through all subject tables for each year and count the number of non-empty data in fourth column
# The fourth column is where the first variable of each table is recorded
for year in years:
    for table in df_tables['Table']:
        # Construct the file path
        file_path = f"../Raw_Data/ACS-{table}-{year}-Data.csv"
        
        try:
            # Try to read the file
            data = pd.read_csv(file_path)
            
            # Count the number of non-empty, numeric rows in the fourth column
            non_empty_numeric_count = data.iloc[:, 3].notnull().sum()
            
            # Append year, table, and count to the temp data list
            temp_data.append({'Year': year, 'Table': table, 'Count': non_empty_numeric_count})
        except FileNotFoundError:
            # If the file does not exist, append year, table name, count as 0, and a note mentioning the table doesn't exist
            temp_data.append({'Year': year, 'Table': table, 'Count': -1})


# Convert the list of dictionaries to a DataFrame
data_counts = pd.DataFrame(temp_data)

# Rearrange DataFrame to have 'Year' and 'Table' as the first columns if needed
data_counts = data_counts[['Year', 'Table'] + [col for col in data_counts.columns if col not in ['Year', 'Table']]]

pivoted_df = data_counts.pivot(index='Table', columns='Year', values='Count')

# Transpose before saving to ensure the layout matches your requirements
pivoted_df.to_csv('subject_tables_availability.csv')

# filter out tables with more than 30,000 values
filtered_df = pivoted_df[pivoted_df.gt(30000).all(axis=1)]
final_df = pd.DataFrame(filtered_df.index.tolist(), columns=['Table'])

final_df.to_csv('final_tables_merged.csv', index=False)

## Final Merge
### Using 'combined_id_E_M.csv', merge 2015 ~ 2022 data for only California Zipcodes (90001 ~ 96162)

In [3]:
import pandas as pd
# Load the 'combined_id_E_M.csv' file, which is combined IDs of 2015-2022
combined_id_df = pd.read_csv('combined_id_E_M.csv')
final_df = pd.read_csv('final_tables_merged.csv')

years = range(2015, 2023)

for table in final_df['Table']:
    
    merged_acs_df = pd.DataFrame()

    for year in years:

        # Filter for IDs starting with table name in current loop for 2015 ~ 2022
        globals()[f"ids_{year}_{table}"] = combined_id_df[f'{year}_ID'][combined_id_df[f'{year}_ID'].str.startswith(f'{table}')].tolist()

        # Load the ACS raw data files for 2015 ~ 2022
        globals()[f"acs_{year}_df"] = pd.read_csv(f'../Raw_Data/ACS-{table}-{year}-Data.csv').iloc[:, 2:]

        globals()[f"acs_{year}_df"] = globals()[f"acs_{year}_df"].rename(columns={'NAME.1': 'ZIP_CODE'})

        # Extract the numerical part from 'ZIP_CODE' column and convert to integers
        zip_code_nums = globals()[f"acs_{year}_df"]['ZIP_CODE'].str.split().str[-1].astype(int)

        # Load the zipcodes from 'zipcodes_cal870.csv'
        zipcodes_cal = pd.read_csv('data_check/zipcodes_cal870.csv')

        # Extract the zip codes from the dataframe
        zip_codes = zipcodes_cal['zipcode'].tolist()

        # Create a mask for the desired zip code range (California) and the zip codes from 'zipcodes_cal870.csv'
        mask = (zip_code_nums >= 90001) & (zip_code_nums <= 96162) & (zip_code_nums.isin(zip_codes))

        # Apply the mask to the dataframe to filter rows
        globals()[f"filtered_acs_{year}_df"] = globals()[f"acs_{year}_df"][mask]

        # Filter for 'ZIP_CODE' and IDs (all the coluns we need for final data)
        globals()[f"columns_{year}"] = ['ZIP_CODE'] + globals()[f"ids_{year}_{table}"]

        # Apply filtering to California-filtered data
        globals()[f"filtered_acs_{year}_df"] = globals()[f"filtered_acs_{year}_df"][globals()[f"columns_{year}"]]

        # Add 'Year' as the first column to denote 2015-2022
        globals()[f"filtered_acs_{year}_df"].insert(0, 'Year', year) 
        
        # To make sure we use the same 2015 columns with 2016-2022
        if year != 2015:
            globals()[f"filtered_acs_{year}_df"].columns = merged_acs_df.columns

        # Vertically merge the two filtered datasets
        merged_acs_df = pd.concat([merged_acs_df, globals()[f"filtered_acs_{year}_df"]], ignore_index=True)
    
    merged_acs_df.to_csv(f'../Merged_Data/Merged_{table}.csv', index=False)